# **Récupération, pré-traitement et analyse de posts Reddit en rapport avec l'entreprise Apple afin de conduire une analyse de sentiments**

L'objectif de cette partie est de **récupérer des posts Reddit** relatifs à Apple afin de les **analyser** et de pouvoir les incorporer à notre **algorithme de trading final** sous forme d'une aide à la décision.

On va d'abord **importer** naïvement les derniers posts relatifs à Apple à l'aide de l'**API de Reddit**. On va ensuite trier ces données, les nettoyer et faire un ensemble de **statistiques descriptives** sur les posts nettoyés. Nous allons d'abord observer un nuage de mots sur les titres des derniers posts. Puis, nous créerons un dataframe regroupant les derniers posts, leur auteur, leur date de publication, leur titre ainsi que leur URL.

Par suite, nous observerons le lien qui peut exister entre le nombre de posts publiés et le cours d'Apple en bourse.

Puis nous déterminerons l'influence de certains auteurs dans le nombre de posts écrits.

Enfin, nous réaliserons des statistiques descriptives sur le dataframe créé initialment.


Tout d'abord, on installe **praw** et **wordcloud** qui seront deux bibliothèques essentielles pour la suite de ce notebook.

In [ ]:
pip install praw wordcloud

On importe aussi toutes les **bibliothèques** qui nous seront utiles :

In [ ]:
import praw
import pandas as pd
import numpy as np
from wordcloud import WordCloud,  STOPWORDS
import matplotlib.pyplot as plt
import re
from collections import Counter

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

On importe également les fonctions codées dans les fichiers précédents et qui nous seront à nouveau utiles ici :

In [ ]:
def cleaning_text(text):
    #Passage du texte en miniscules
    text=text.lower()
    #Suppression des chiffres
    text=re.sub(r'\d+', '', text)
    #Suppresion de /r/
    text =re.sub(r'/r/', '', text)
    #Suppression de la ponctuation et des symboles spéciaux
    text=re.sub(r'[^\w\s]', '', text)
    return text


def count_common_words(text):
    words=text.split()
    dict={}
    for el in words :
        if el not in dict:
            dict[el]=1
        else:
            dict[el]+=1
    return dict

def most_common_words(dictionary):
    sorted_dict=dict(sorted(dictionary.items(), key=lambda item: item[1], reverse=True))
    return sorted_dict



def combine_dictionaries(df,df_column_name):
    combined_dict={}
    column_index=df.columns.get_loc(str(df_column_name))
    for i in range(len(df)):
        temp_dictionary=df.iloc[i,int(column_index)]

        for key, value in temp_dictionary.items():
            if key not in combined_dict:
                combined_dict[key]=value
            else:
                combined_dict[key]+=value
    return combined_dict

stops = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
def clean_stopwords(text):
    words = word_tokenize(text)
    cleaned_text=[word for word in words if word not in stops]
    return ' '.join(cleaned_text)

# **I/ Récupération des articles sous forme d'un dataframe**

Nous allons d'abord observer les mots les plus utilisés dans les titres des 500 derniers posts relatifs à Apple. Puis, nous allons créer et nettoyer un dataframe regroupant les informations principales concernants les posts concernant Apple. Pour ce faire, nous utilisons l'API de Reddit. Nous avons,préalablement, créé un profit "programmeur" sur Reddit. Ce profit permet d'obtenir l'identification client et le mot de passe, qui donnent la possibilité d'importer les données de Reddit.

## 1/ Nuage de mots des 500 derniers posts sur Apple

On fait les configurations nécessaires à l'utilisation de PRAW pour la récupération des posts:

In [ ]:
reddit = praw.Reddit(client_id='xIq0ALkJ0RWzM5ZLwwiQKA',
                     client_secret='DeHliktGK8nfhDXsJFiebqgeZKhHXQ',
                     user_agent='Matlpg')

On récupère les 500 premiers posts du subreddit 'apple' :

In [ ]:
subreddit = reddit.subreddit('apple')
top_posts = subreddit.new(limit=500) #On récupère ainsi les 500 premiers posts Reddit sur Apple

In [ ]:
#Création d'un dataframe 
posts_data = []
for post in top_posts:
    text = post.selftext if post.selftext else "Text Not Available"
    date = pd.to_datetime(post.created_utc, unit='s')
    post_data = {
        "Titre": post.title,
        "Auteur": str(post.author),
        "Texte": text,
        "Date": date,
        "url": post.url
    }
    posts_data.append(post_data)

df = pd.DataFrame(posts_data)

On nettoie le texte récupéré :

In [ ]:
df['Cleaned_text']=df['Texte'].apply(cleaning_text)
df['Cleaned_text']=df['Cleaned_text'].apply(clean_stopwords)

df.head()

In [ ]:
text=' '.join(df['Cleaned_text'])
words=text.split()
word_counts=Counter(words)
wordcloud = WordCloud(width = 800, height = 500, background_color ='white').generate_from_frequencies(word_counts)

plt.figure(figsize = (8, 8)) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.show()

"**Advice thread**" revient très souvent. En effet, on remarque en analysant le dataframe, qu'un user publie **le même post** chaque jour avec à chaque fois ce titre. Nous pouvons l'enlever. 
Par ailleurs, on constate que de nombreux articles ne contiennent pas de texte. Ces derniers contiennent généralement un **lien** vers un article de presse extérieur à Reddit. Afin de récupérer du contenu plus intéressant, nous allons récupérer les **commentaires principaux** sous chacun des posts récupérés. Ce sont eux qui seront analysés par notre modèle de NLP.

## Récupération des commentaires principaux :

On supprime les posts avec un "text not available"

In [ ]:
df = df[df['text_cleaned'].str.lower() != "text not available"]
df.reset_index(drop=True, inplace=True)

On récupère les commentaires principaux.

In [ ]:
def extract_submission_id(url):
    # Utilisation d'une expression régulière pour extraire l'ID de la soumission
    match = re.search(r'/comments/(\w+)/', url)
    # Si l'expression régulière trouve un match, retourner l'ID, sinon retourner None
    return match.group(1) if match else None

def get_top_comments(url):
    submission_id = extract_submission_id(url)
    
    # Vérifier si submission_id est valide
    if not submission_id:
        print(f"L'ID de la soumission extrait de l'URL '{url}' est invalide.")
        return []
    
    # Si submission_id est valide, continuer à récupérer les commentaires
    submission = reddit.submission(id=submission_id)
    submission.comment_sort = 'top'
    submission.comments.replace_more(limit=0)
    
    top_comments = []
    for comment in submission.comments[:5]:  # Prendre les 5 premiers commentaires
        top_comments.append(comment.body)
    
    return top_comments


for index, row in df.iterrows():
    top_comments = get_top_comments(row['url'])
    for i, comment in enumerate(top_comments):
        df.loc[index, f'comment_{i+1}'] = comment

On nettoie le texte des commentaires principaux récupérés

In [ ]:
df['comment_1_clean'] = df['comment_1'].apply(cleaning_text)
df['comment_2_clean'] = df['comment_2'].apply(cleaning_text)
df['comment_3_clean'] = df['comment_3'].apply(cleaning_text)
df['comment_4_clean'] = df['comment_4'].apply(cleaning_text)
df['comment_5_clean'] = df['comment_5'].apply(cleaning_text)

df.head()

On supprime les posts "Advice Thread"

In [ ]:
df = df[~df['text_cleaned'].str.contains("daily advice thread", case=False, na=False)]

# Analyse descriptive du dataframe

Après avoir obtenu un dataframe nettoyé regroupant les 500 derniers posts Reddit relatifs à Apple, nous allors désormais utiliser ce dataframe pour tenter d'observer un lien entre le cours boursier d'Apple et le nombre de posts publiés. Puis nous observerons l'impact qu'ont certains utilisateurs sur le nombre de posts publiés. Enfin, nous réaliserons des statistiques descriptives sur les posts Reddit.

## A/ Lien entre le cours d'Apple et le nombre de posts écrits

Dans cette partie, nous allons observer s'il existe, a priori, un lien entre le nombre de posts publiés sur Reddit et relatifs à Apple et le cours d'Apple en bourse.

Tout d'abord, nous allons tracer un graphe regroupant, en abscisse, le temps et, en ordonnée, le nombre de posts cumulé publiés sur Reddit.

In [ ]:
df['count'] = 1  #Cette colonne sert à compter les occurences
df_grouped = df.groupby(df['Date'].dt.date)['count'].sum().cumsum()

#On crée le graphique
plt.figure(figsize=(14, 7))
plt.plot(df_grouped.index, df_grouped.values, marker='o', linestyle='-')
plt.title('Cumulative Number of Posts Over Time')
plt.xlabel('Date')
plt.ylabel('Cumulative Number of Posts')
plt.grid(True)
plt.tight_layout()
plt.show()

Nous observons une courbe **quasi linéaire**, avec quelques fluctuations. Nous allons observer si ces fluctuations peuvent avoir un **lien** avec le cours d'Apple en bourse.

Superposons la courbe obtenue précédemment avec **la courbe de l'indice d'Apple en bourse**. Ainsi, nous pourrons observer s'il peut y avoir un lien entre l'indice d'Apple en bourse et le nombre de posts écrits sur Reddit

In [ ]:
import yfinance as yf

# Trouver la date du post le plus ancien
oldest_post_date = df['date'].min()

# Utiliser cette date pour télécharger les données historiques d'Apple
apple_ticker = 'AAPL'
apple_data = yf.download(apple_ticker, start=oldest_post_date)

df_apple_aligned = apple_data.reindex(df['date'].unique(), method='ffill')


# Créer le graphe avec deux axes y
fig, ax1 = plt.subplots(figsize=(14, 7))

# Axe pour le nombre de posts
ax1.set_xlabel('Date')
ax1.set_ylabel('Cumulative Number of Posts', color='tab:blue')
ax1.plot(df_grouped.index, df_grouped.values, marker='o', linestyle='-')
ax1.tick_params(axis='y', labelcolor='tab:blue')
ax1.grid(True)

# Instantier un second axe y qui partage le même axe x
ax2 = ax1.twinx()
ax2.set_ylabel('Stock Price', color='tab:red')
ax2.plot(df_apple_aligned.index, df_apple_aligned['Close'], color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')

# Titre et style
plt.title('Cumulative Number of Apple-related Posts Over Time vs Apple Stock Price')
fig.tight_layout()

plt.show()

Graphiquement, nous n'observons **pas de lien direct évident** entre le nombre de posts publiés et le cours d'Apple. Néanmoins, il semblerait qu'un plus grand nombre de posts soient écrits juste avant un choc du cours boursier d'Apple.

Nous allons maintenant **calculer le coefficient de correlation**, au sens de Pearson, entre le nombre de posts Reddit relatifs à Apple publiés et le cours boursier d'Apple. 

In [ ]:
# On réindexe le dataframe
df_apple_aligned.reset_index(inplace=True)
df_apple_aligned.rename(columns={'index': 'Date'}, inplace=True)
df_apple_aligned
# On extrait de df_apple_aligned un dataframe composé d'une colonne avec la date et le cours moyen d'Apple à cette date

# On convertit la colonne 'datetime' en type datetime
df_apple_aligned['Date'] = pd.to_datetime(df_apple_aligned['Date'])

# On extrait la date sans l'heure
df_apple_aligned['date2'] = df_apple_aligned['Date'].dt.date

# On groupe par date et calculer la moyenne du cours boursier
daily_avg_stock_data = df_apple_aligned.groupby('date2')['Close'].mean().reset_index()

# On fait de même pour df_sorted
df['date'] = pd.to_datetime(df['date'])

# On extrait la date sans l'heure
df['date2'] = df['date'].dt.date

# On groupe par date et calculer la moyenne du cours boursier
daily_post_count= df.groupby('date2')['count'].count().reset_index()

# On calcule le coefficient de correlation
from scipy.stats import pearsonr

# On fusionne les deux ensembles de données sur la date
merged_data = pd.merge(daily_post_count, daily_avg_stock_data, on='date2')

# On calcule la corrélation de Pearson
correlation, p_value = pearsonr(merged_data['count'], merged_data['Close'])
correlation

Nous observons que le coefficient de correlation est **beaucoup plus faible**, en valeur absolu que 1. Donc le nombre de post publiés sur Reddit ne semble que **peu corrélé** au cours boursier. En réalité, ce sont peut-être les sentiments dégagés dans ces posts et dans les commentaires qui peuvent être corrélés avec le cours boursier d'Apple. 

## B/Lien entre les auteurs des posts et le nombre de posts écrits

Nous allons maintenant déterminer **les plus gros contributeurs sur Reddit**. Pour ce faire, nous allons d'abord créer un dataframe avec l'ensemble des contributeurs du /r/Apple. Puis nous réaliserons des statistiques sur ces contributeurs, comme leur influence sur l'ensemble des posts Reddit publiés.

D'abord, crée un dataframe regroupant l'ensemble des contributeurs du subreddit /r/Apple

In [ ]:
df_authors = df.groupby('auteur').agg(
    number_of_posts=pd.NamedAgg(column='Titre', aggfunc='count'),
    latest_post_date=pd.NamedAgg(column='date', aggfunc='max')
).reset_index()

# Tri des auteurs par le nombre d'articles écrits, en ordre décroissant
df_authors_sorted = df_authors.sort_values(by='number_of_posts', ascending=False).reset_index(drop=True)

df_authors_sorted.head()

Nous observons que *favicondotico* et *AutoModerator* sont les plus gros contributeurs du lien Reddit sur Apple. Nous allons observer l'influence de ces contributeurs dans le nombre de posts publiés. Nous allons d'abord extraire de sorted un Dataframe avec seulement les posts de *favicondotico* et de *AutoModerator*. Puis nous allons tracer sur un même graphe le nombre de posts, en cumulé, de *AutoModerator* et de *favicondotico*.

In [ ]:
df_favicondotico = df[(df['auteur'] == 'favicondotico') | (df['auteur'] == 'AutoModerator')]
df_favicondotico = df_favicondotico.reset_index(drop=True)


df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df['cumulative_posts'] = df['count'].cumsum()

# Pour df_favicondotico_reset, nous faisons la même chose
df_favicondotico['date'] = pd.to_datetime(df_favicondotico['date'])
df_favicondotico.set_index('date', inplace=True)
df_favicondotico['cumulative_posts_favicondotico'] = df_favicondotico['count'].cumsum()

# Création du graphique
fig, ax1 = plt.subplots(figsize=(14, 7))

# Tracer le nombre cumulatif de tous les posts
ax1.plot(df.index, df['cumulative_posts'], label='Cumulative Posts (All Authors)', color='tab:blue')
ax1.set_xlabel('Date')
ax1.set_ylabel('Cumulative Posts (All Authors)', color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')

# Créer un second axe pour le nombre cumulatif de posts par favicondotico
ax2 = ax1.twinx()
ax2.plot(df_favicondotico.index, df_favicondotico['cumulative_posts_favicondotico'], label='Cumulative Posts (favicondotico)', color='tab:red')
ax2.set_ylabel('Cumulative Posts (favicondotico, AutoModerator)', color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')

# Légendes et titre
fig.tight_layout()
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
plt.title('Cumulative Posts Over Time')

# Afficher le graphique
plt.show()

In [ ]:
df.reset_index(inplace=True)
df_favicondotico.reset_index(inplace=True)

df.rename(columns={'index': 'date'}, inplace=True)
df_favicondotico.rename(columns={'index': 'date'}, inplace=True)

Nous observons que le nombre de posts publiés par *favicondotico* et *AutoModerator* a **une très grande influence** sur le nombre de posts publiés au total. En effet, la tendance du nombre de posts publiés sur le subreddit /r/Apple est la même que la tendance du nombre de posts publiés par *favicondotico* ou *AutoModerator*

Nous allons visualiser la proportion de participation aux posts de chaque contributeur à l'aide d'un **diagramme en camembert**.

In [ ]:
# Calculer le total des posts
total_posts = df_authors_sorted['number_of_posts'].sum()

# Calculer le pourcentage des posts totaux pour chaque auteur
df_authors_sorted['percentage'] = (df_authors_sorted['number_of_posts'] / total_posts) * 100

# Filtrer pour inclure seulement les auteurs avec plus de 1% des posts
df_filtered = df_authors_sorted[df_authors_sorted['percentage'] > 1]

# Couleurs en nuances de jaune
colors = ['gold', 'yellow', 'lightyellow', 'lemonchiffon', 'khaki', 'darkkhaki', 'palegoldenrod']

# Tracer le diagramme en camembert
plt.figure(figsize=(10, 8))
plt.pie(df_filtered['percentage'], labels=df_filtered['auteur'], autopct='%1.1f%%',colors=colors, startangle=140)
plt.axis('equal')  # S'assurer que le camembert est bien circulaire
plt.show()

Nous observons que *favicondotico* et *AutoModerator* contribuent **à plus de la moitié** des posts complets. D'où la grande influence que ces deux contributeurs ont sur l'ensemble des posts publiés. Nous comprenons désormais pourquoi la tendance concernant le nombre total de posts publiés est très proche de celle concernant le nombre total de posts publiés par *favicondotico* et *AutoModerator*.

## C/ Statistiques sur les mots les plus utilisés

L'objectif de cette partie est de réaliser des **statistiques descriptives sur l'occurence de certains mots** dans les textes écrits dans les dataframes.

En partie II/A nous avons réalisé un nuage de mots permettant de rendre compte des mots les plus utilisés dans les titres des articles. On va maintenant créer un dataframe qui compte l'occurence des mots identifiés dans la section II/A

On ajoute le texte compilé des 5 commentaires les plus appréciés sur chaque post et on nettoie ce texte.

In [ ]:
def get_top_comments_text_join(url, limit=5):
    # Utiliser la fonction extract_submission_id pour obtenir l'identifiant du post
    submission_id = extract_submission_id(url)
    # Vérifier que submission_id n'est pas None
    if submission_id:
        submission = reddit.submission(id=submission_id)

        # Récupère et trie les commentaires par score
        submission.comment_sort = 'top'
        submission.comments.replace_more(limit=5)  # Charge tous les commentaires
        top_comments = list(submission.comments[:limit])

        # Compile le texte des commentaires
        comments_text = ' '.join(comment.body for comment in top_comments if hasattr(comment, 'body'))
        return comments_text
    else:
        return "No valid ID found in URL"
# Ajouter une nouvelle colonne avec le texte des commentaires compilés
df['top_comments_text_compiled'] = df['url'].apply(get_top_comments_text_join)
df['top_comments_clean_compiled']=df['top_comments_text_compiled'].apply(cleaning_text)

# On supprime aussi les colonnes "Count" et "Cumulative_count"
df = df.drop('count', axis=1)
df = df.drop('cumulative_count', axis=1)

Nous pouvons désormais réaliser une analyse descriptive du dataframe. On va déterminer l'occurence des mots observés dans le nuage de mots précédent. Pour ce faire nous allons créer une colonne "count_mot" pour connaitre l'occurence d'un mot dans le post principal et une colonne "count_mot2" pour connaitre son occurence dans les commentaires.

In [ ]:
df['apple_count'] = df['Cleaned_text'].str.lower().str.count('apple')
df['iphone_count'] = df['Cleaned_text'].str.lower().str.count('iphone')
df['airpods_count'] = df['Cleaned_text'].str.lower().str.count('airpods')
df['iphonepro_count'] = df['Cleaned_text'].str.lower().str.count('iphone pro')
df['android_count'] = df['Cleaned_text'].str.lower().str.count('android')
df['ios_count'] = df['Cleaned_text'].str.lower().str.count('ios')
df['ipad_count'] = df['Cleaned_text'].str.lower().str.count('ipad')
df['ipadpro_count'] = df['Cleaned_text'].str.lower().str.count('ipad pro')
df['samsung_count'] = df['Cleaned_text'].str.lower().str.count('samsung')

df['apple_count2'] = df['top_comments_clean_compiled'].str.lower().str.count('apple')
df['iphone_count2'] = df['top_comments_clean_compiled'].str.lower().str.count('iphone')
df['airpods_count2'] = df['top_comments_clean_compiled'].str.lower().str.count('airpods')
df['iphonepro_count2'] = df['top_comments_clean_compiled'].str.lower().str.count('iphone pro')
df['android_count2'] = df['top_comments_clean_compiled'].str.lower().str.count('android')
df['ios_count2'] = df['top_comments_clean_compiled'].str.lower().str.count('ios')
df['ipad_count2'] = df['top_comments_clean_compiled'].str.lower().str.count('ipad')
df['ipadpro_count2'] = df['top_comments_clean_compiled'].str.lower().str.count('ipad pro')
df['samsung_count2'] = df['top_comments_clean_compiled'].str.lower().str.count('samsung')


Nous supprimons également tous les textes non valides.

In [ ]:
df = df[df['top_comments_cleaned'].str.lower() != 'no valid id found in url']
df.reset_index(drop=True, inplace=True)

Nous utilisons la fonction "describe" pour obtenir un tableau relatif aux statistiques sur l'occurence des mots sélectionnés plus haut.

In [ ]:
round(df.describe(), 3)

 Au total, le mot "Apple" n'apparait que peu de fois par article. Cela est sûrement dû au fait que les contributeurs parlent régulièrement des produits d'Apple, et non pas d'Apple même. En effet, les contributeurs semblent plus aborder le thème des produits comme l'Iphone, l'Ipad, les airpods que d'Apple en lui-même. On remarque que le mot "Android" est très rarement utilisé également. Ainsi, les posts sur Apple ne parle que peu du système d'exploitation concurrent à IOS. Une limite à ce modèle est la présence des posts du type "daily thread". En effet, ces posts ne sont pas d'un grand intérêt car le contributeur écrit chaque jour le même post. Nous allons donc réaliser des statistques descriptives après avoir supprimé tous les "daily threads".

 Pour ce qui est des commentaires, Apple apparait 2 fois en moyenne sur l'ensemble des posts tandis qu'Iphone apparait 1.5 fois en moyenne. La conclusion que nous faisons est la même que précédemment : les contributeurs discutent principalement des produits d'Apple et assez peu, voire pas du tout, de la concurrence d'Apple.

Nous allons maintenant nettoyer le tableau pour l'implémenter dans l'algorithme de trading.

In [ ]:
cols_to_drop = [col for col in df.columns if col.endswith('_count')]
df.drop(cols_to_drop, axis=1, inplace=True)
cols_to_drop2 = [col for col in df.columns if col.endswith('_count2')]
df.drop(cols_to_drop2, axis=1, inplace=True)